In [1]:
import pandas as pd
import numpy as np
from scipy import stats 

In [2]:
df = pd.read_csv('rappler.csv')

In [3]:
df = df[df['Loc'].notnull()]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 161 to 200
Data columns (total 7 columns):
source_id    40 non-null int64
date         40 non-null object
Loc          40 non-null object
lat          39 non-null float64
long         39 non-null float64
status       25 non-null object
counts       40 non-null int64
dtypes: float64(2), int64(2), object(3)
memory usage: 2.5+ KB


In [5]:
def parse(df):
    df['date'] = pd.to_datetime(df['date'])
    print(df.info())
    
    # Get min/max/mean values
    dfa = pd.pivot_table(df, values = 'counts', index=['date', 'Loc'], columns='status', aggfunc=[min, max, np.mean, stats.mode])
    
    # Remove multi-index
    dfa.columns = ["_".join(pair) for pair in dfa.columns]
    dfa = dfa.reset_index()
    
    # Replace 0 with np.nan to forward fill null values
    dfa = dfa.replace(0, np.nan)
    
    # Forward filling needs to be by area
    places = list(df['Loc'].unique())
    
    global dfb
    dfb = pd.DataFrame()
    for place in places:
        df_temp = dfa[dfa['Loc'] == place].fillna(method='ffill')
        dfb = dfb.append(df_temp)
    return dfb

In [6]:
parse(df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 161 to 200
Data columns (total 7 columns):
source_id    40 non-null int64
date         40 non-null datetime64[ns]
Loc          40 non-null object
lat          39 non-null float64
long         39 non-null float64
status       25 non-null object
counts       40 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 2.5+ KB
None


,date,Loc,min_confirmed,min_suspected,max_confirmed,max_suspected,mean_confirmed,mean_suspected,mode_confirmed,mode_suspected
2,2020-01-23,cebu,NaN,NaN,NaN,555.0,NaN,185.000000,NaN,"([0], [2])"
3,2020-01-24,cebu,NaN,14.0,NaN,881.0,NaN,447.500000,NaN,"([14], [1])"
8,2020-01-30,cebu,NaN,14.0,NaN,881.0,NaN,447.500000,NaN,"([0], [2])"
11,2020-01-31,cebu,NaN,14.0,NaN,100.0,NaN,33.333333,NaN,"([0], [2])"
14,2020-02-01,cebu,NaN,14.0,NaN,100.0,NaN,33.333333,NaN,"([0], [1])"
15,2020-02-02,cebu,1.0,14.0,1.0,100.0,1.0,33.333333,"([1], [1])","([0], [2])"
16,2020-02-05,cebu,1.0,14.0,1.0,100.0,1.0,33.333333,"([1], [1])","([0], [2])"
1,2020-01-23,aklan,NaN,3.0,NaN,3.0,NaN,3.000000,NaN,"([3], [1])"
4,2020-01-25,aklan,NaN,80.0,NaN,80.0,NaN,80.000000,NaN,"([80], [1])"
6,2020-01-28,aklan,NaN,11.0,NaN,11.0,NaN,11.000000,NaN,"([11], [1])"
